In [597]:
import pandas as pd
import numpy as np

In [599]:
df=pd.read_csv('Fraud Detection Dataset.csv')

In [601]:
df.head()

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Time_of_Transaction,Device_Used,Location,Previous_Fraudulent_Transactions,Account_Age,Number_of_Transactions_Last_24H,Payment_Method,Fraudulent
0,T1,4174,1292.76,ATM Withdrawal,16.0,Tablet,San Francisco,0,119,13,Debit Card,0
1,T2,4507,1554.58,ATM Withdrawal,13.0,Mobile,New York,4,79,3,Credit Card,0
2,T3,1860,2395.02,ATM Withdrawal,NaN,Mobile,NaN,3,115,9,NaN,0
3,T4,2294,100.10,Bill Payment,15.0,Desktop,Chicago,4,3,4,UPI,0
4,T5,2130,1490.50,POS Payment,19.0,Mobile,San Francisco,2,57,7,Credit Card,0


In [603]:
# CHECKING FOR NULL VALUES 

In [605]:
df.isnull().sum()

Transaction_ID                         0
User_ID                                0
Transaction_Amount                  2520
Transaction_Type                       0
Time_of_Transaction                 2552
Device_Used                         2473
Location                            2547
Previous_Fraudulent_Transactions       0
Account_Age                            0
Number_of_Transactions_Last_24H        0
Payment_Method                      2469
Fraudulent                             0
dtype: int64

In [607]:
df['Transaction_Amount'].fillna(df['Transaction_Amount'].median(), inplace=True)
df['Time_of_Transaction'].fillna(df['Time_of_Transaction'].mean(), inplace=True)
df['Device_Used'].fillna(df['Device_Used'].mode()[0], inplace=True)
df['Location'].fillna(df['Location'].mode()[0], inplace=True)
df['Payment_Method'].fillna(df['Payment_Method'].mode()[0], inplace=True)
df['Fraudulent'].fillna(df['Fraudulent'].mode()[0], inplace=True)
import warnings
warnings.filterwarnings('ignore')


In [609]:
# using pandasprofiling for eda 

from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Fraud Detection Data Report", explorative=True)
profile.to_notebook_iframe()








Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|███████████████████████████████████████████| 12/12 [00:00<00:00, 24.81it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [611]:
df.drop('Transaction_ID', axis=1, inplace=True)
df.drop('User_ID', axis=1, inplace=True)
df.drop('Location', axis=1, inplace=True)


In [613]:
correlation=df.select_dtypes(include='number').corr()
print(correlation['Fraudulent'].sort_values(ascending=False))

Fraudulent                          1.000000
Time_of_Transaction                 0.006845
Account_Age                         0.006203
Transaction_Amount                  0.005359
Previous_Fraudulent_Transactions    0.001136
Number_of_Transactions_Last_24H    -0.003877
Name: Fraudulent, dtype: float64


In [615]:
df['Device_Used'].value_counts()

Device_Used
Desktop           18268
Mobile            15614
Tablet            15588
Unknown Device     1530
Name: count, dtype: int64

In [617]:
df['Payment_Method'].value_counts()


Payment_Method
UPI               14356
Debit Card        11802
Net Banking       11666
Credit Card       11646
Invalid Method     1530
Name: count, dtype: int64

In [619]:
df['Transaction_Type'].value_counts()

Transaction_Type
Bill Payment       10340
Bank Transfer      10276
ATM Withdrawal     10164
POS Payment        10126
Online Purchase    10094
Name: count, dtype: int64

In [621]:
# OneHot Encoding

df["Transaction_Type"] = df["Transaction_Type"].map({"Bill Payment":1, "Bank Transfer": 2, 
                                 "ATM Withdrawal": 3, "POS Payment": 4,
                                 "Online Purchase": 5})
df["Device_Used"] = df["Device_Used"].map({"Desktop": 1, "Mobile": 2, 
                                 "Tablet": 3, "Unknown Device": 4})
df["Payment_Method"] = df["Payment_Method"].map({"UPI": 1, "Debit Card": 2, 
                                 "Net Banking": 3, "Credit Card": 4,"Invalid Method":5})



In [623]:
df['Fraudulent']=df['Fraudulent'].map({0:"No Fraud",1:"Fraud"})
df.head()

,Transaction_Amount,Transaction_Type,Time_of_Transaction,Device_Used,Previous_Fraudulent_Transactions,Account_Age,Number_of_Transactions_Last_24H,Payment_Method,Fraudulent
0,1292.76,3,16.0000,3,0,119,13,2,No Fraud
1,1554.58,3,13.0000,2,4,79,3,4,No Fraud
2,2395.02,3,11.4884,2,3,115,9,1,No Fraud
3,100.10,1,15.0000,1,4,3,4,1,No Fraud
4,1490.50,4,19.0000,2,2,57,7,4,No Fraud


In [625]:
print(df.isnull().sum())


Transaction_Amount                  0
Transaction_Type                    0
Time_of_Transaction                 0
Device_Used                         0
Previous_Fraudulent_Transactions    0
Account_Age                         0
Number_of_Transactions_Last_24H     0
Payment_Method                      0
Fraudulent                          0
dtype: int64


In [642]:
from sklearn.model_selection import train_test_split
x=np.array(df[['Time_of_Transaction','Account_Age','Transaction_Amount','Previous_Fraudulent_Transactions','Transaction_Type','Device_Used','Payment_Method']])
y=np.array(df['Fraudulent'])
xtrain, xtest, ytrain, ytest = train_test_split(x, y,test_size=0.2, random_state=42)
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(xtrain,ytrain)
ypred=lr.predict(xtest)


In [644]:
from sklearn.metrics import accuracy_score
accuracy_score(ypred,ytest)

0.9498039215686275

In [646]:

from sklearn.tree import DecisionTreeClassifier
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)
print(model.score(xtest, ytest))




0.8945098039215686


In [648]:
#features = ['Time_of_Transaction','Account_Age','Transaction_Amount','Previous_Fraudulent_Transactions','Transaction_Type','Device_Used','Payment_Method']
features=np.array([[5,20,2000,3,2,3,1]])

In [650]:
lr.predict(features)

array(['No Fraud'], dtype=object)

In [652]:
model.predict(features)

array(['No Fraud'], dtype=object)